In [203]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.colors as mcolors

**Generación de insights para ofertas relámpago:**

En conjunto con el desafío te compartimos un archivo llamado ofertas_relampago.csv el cual posee información de los resultados de ofertas del tipo relámpago para un periodo de tiempo y un país determinado.


Es decir, son ofertas que tienen una duración definida de algunas horas y un porcentaje de unidades (stock) comprometidas.

El objetivo de este desafío es hacer un EDA sobre estos datos buscando insights sobre este tipo de ofertas.

Las columnas del dataset son autoexplicativas pero puedes preguntarnos cualquier duda.

# 1. Importar los datos

In [164]:
ofertas_relampagos_df = pd.read_csv('../data/ofertas_relampago.csv')

In [165]:
ofertas_relampagos_df.OFFER_START_DATE = pd.to_datetime(ofertas_relampagos_df.OFFER_START_DATE)
ofertas_relampagos_df.OFFER_START_DTTM = pd.to_datetime(ofertas_relampagos_df.OFFER_START_DTTM)
ofertas_relampagos_df.OFFER_FINISH_DTTM = pd.to_datetime(ofertas_relampagos_df.OFFER_FINISH_DTTM)

In [166]:
ofertas_relampagos_df.SOLD_QUANTITY = ofertas_relampagos_df.SOLD_QUANTITY.fillna(0)
ofertas_relampagos_df.loc[ofertas_relampagos_df.SOLD_QUANTITY==0,'SOLD_AMOUNT'] = 0

In [167]:
ofertas_relampagos_df['SOLD_QUANTITY'] = ofertas_relampagos_df['SOLD_QUANTITY'].fillna(ofertas_relampagos_df.INVOLVED_STOCK-ofertas_relampagos_df.REMAINING_STOCK_AFTER_END)

In [168]:
ofertas_relampagos_df.sample(10)

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID
37458,2021-07-22,2021-07-22 13:00:00+00:00,2021-07-22 13:00:00+00:00,lightning_deal,5,5,0.00,0.0,NaN,free_shipping,APPAREL ACCESORIES,APP & SPORTS,MLM-RINGS
679,2021-06-22,2021-06-22 17:00:00+00:00,2021-06-23 01:00:00+00:00,lightning_deal,3,3,0.00,0.0,A,none,BABY,T & B,MLM-BABY_PLATES
31557,2021-07-25,2021-07-25 07:00:00+00:00,2021-07-25 13:00:03+00:00,lightning_deal,5,4,18.36,1.0,NaN,free_shipping,PERSONAL CARE,BEAUTY & HEALTH,MLM-EYESHADOWS
30652,2021-07-11,2021-07-11 07:00:00+00:00,2021-07-11 12:36:50+00:00,lightning_deal,5,0,2.68,1.0,NaN,none,PERSONAL CARE,BEAUTY & HEALTH,MLM-MASCARAS
1473,2021-07-08,2021-07-08 19:00:00+00:00,2021-07-09 03:00:00+00:00,lightning_deal,9,9,0.00,0.0,A,free_shipping,TOYS AND GAMES,T & B,MLM-ACTION_FIGURES
21786,2021-07-28,2021-07-28 19:00:00+00:00,2021-07-29 01:00:03+00:00,lightning_deal,5,2,3.95,3.0,NaN,none,APPAREL ACCESORIES,APP & SPORTS,MLM-WRISTWATCHES
46296,2021-07-16,2021-07-16 19:00:00+00:00,2021-07-17 01:00:06+00:00,lightning_deal,18,17,1.09,1.0,A,none,FOODS,CPG,MLM-CANDIES
28944,2021-06-10,2021-06-10 13:00:00+00:00,2021-06-10 21:00:00+00:00,lightning_deal,15,15,2.17,1.0,A,none,HOME&DECOR,HOME & INDUSTRY,MLM-MUGS
13098,2021-07-30,2021-07-30 07:00:00+00:00,2021-07-30 13:00:06+00:00,lightning_deal,15,14,9.18,2.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-WALL_AND_CEILING_LIGHTS
12158,2021-07-30,2021-07-30 13:00:00+00:00,2021-07-30 19:00:03+00:00,lightning_deal,5,4,39.76,1.0,NaN,free_shipping,APPAREL ACCESORIES,APP & SPORTS,MLM-WRISTWATCHES


# 2. Creación de nuevas variables relevantes

In [169]:
ofertas_relampagos_df['DURATION_HRS']=((ofertas_relampagos_df.OFFER_FINISH_DTTM-ofertas_relampagos_df.OFFER_START_DTTM).dt.seconds/3600).astype(int)
ofertas_relampagos_df['DAY_OF_WEEK']=ofertas_relampagos_df.OFFER_START_DATE.dt.day_of_week.replace({0:'Lunes',1:'Martes',2:'Miércoles',3:'Jueves',4:'Viernes',5:'Sábado',6:'Domingo'})

In [170]:
ofertas_relampagos_df['PRICE_PER_UNIT'] = 0.0
ofertas_relampagos_df.loc[ofertas_relampagos_df.SOLD_QUANTITY>0,'PRICE_PER_UNIT'] = ofertas_relampagos_df.loc[ofertas_relampagos_df.SOLD_QUANTITY>0,'SOLD_AMOUNT']/ofertas_relampagos_df.loc[ofertas_relampagos_df.SOLD_QUANTITY>0,'SOLD_QUANTITY']
ofertas_relampagos_df['PRICE_PER_UNIT'] = np.round(ofertas_relampagos_df['PRICE_PER_UNIT'],2)

In [171]:
ofertas_relampagos_df.head()

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,DURATION_HRS,DAY_OF_WEEK,PRICE_PER_UNIT
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS,7,Martes,0.79
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,0.00,0.0,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS,6,Martes,0.00
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS,6,Martes,3.58
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,6,Martes,3.52
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,2,Martes,2.64


# 3. EDA

Preguntas
- Histograma de Sold Quantity!!! Duración en Horas
- Por day of the week
- Value counts de VERTICAL!!
- ¿Cuál es la relación entre la duración en horas y la sold quantity? Y por VERTICAL? Por DOM_DOMAIN_AGG1?

Scattergraph supongo??

- ¿Cuál es la relación entre sold_quantity/sold_amount y shipping_payment_type?

## Histogramas

In [172]:
sold_quantity_sin_max_quartil =ofertas_relampagos_df.loc[(ofertas_relampagos_df.SOLD_QUANTITY<=np.quantile(ofertas_relampagos_df.SOLD_QUANTITY,0.75))&(ofertas_relampagos_df.SOLD_QUANTITY>0),'SOLD_QUANTITY']
sold_quantity_sin_max_quartil = sold_quantity_sin_max_quartil.rename('Sold Quantity')
fig = go.Figure(data=go.Histogram(x=sold_quantity_sin_max_quartil,marker_color='#2e3273',nbinsx=7))
fig.update_layout(title_text='Histograma de cantidades vendidas (min. 75%)', title_x=0.5,width=800,height=600,xaxis_title='Sold Quantity',yaxis_title='Frecuencia')

In [173]:
price_per_unit_sin_max_quartil =ofertas_relampagos_df.loc[(ofertas_relampagos_df.PRICE_PER_UNIT<=np.quantile(ofertas_relampagos_df.PRICE_PER_UNIT,0.75))&(ofertas_relampagos_df.SOLD_QUANTITY>0),'PRICE_PER_UNIT']
price_per_unit_sin_max_quartil = price_per_unit_sin_max_quartil.rename('Precio unitario')
fig = go.Figure(data=go.Histogram(x=price_per_unit_sin_max_quartil,marker_color='#2e3273',nbinsx=7))
fig.update_layout(title_text='Histograma de precio unitario de ofertas vendidas (min. 75%)', title_x=0.5,width=800,height=600,xaxis_title='Precio unitario',yaxis_title='Frecuencia')

#ofertas_relampagos_df.loc[ofertas_relampagos_df.PRICE_PER_UNIT<=np.quantile(ofertas_relampagos_df.PRICE_PER_UNIT,0.75),'PRICE_PER_UNIT'].plot(kind='hist')

In [174]:
duracion_hrs =ofertas_relampagos_df.loc[:,'DURATION_HRS']
duracion_hrs = duracion_hrs.rename('Duración')
fig = go.Figure(data=go.Histogram(x=duracion_hrs,marker_color='#2e3273',nbinsx=10))
fig.update_layout(title_text='Histograma de duración de ofertas', title_x=0.5,width=800,height=600,xaxis_title='Duración (hrs)',yaxis_title='Frecuencia')

## Gráficos de barra

In [175]:
dia_semana = ofertas_relampagos_df.groupby('DAY_OF_WEEK').agg({'OFFER_TYPE':'count','SOLD_QUANTITY':'mean','SOLD_AMOUNT':'mean'})
dia_semana = dia_semana.loc[['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']]
fig =  make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=dia_semana.index,y=dia_semana['OFFER_TYPE'],marker_color='#2e3273',name='Ofertas totales'))
fig.add_trace(go.Scatter(x=dia_semana.index,y=dia_semana['SOLD_AMOUNT'],marker_color='#fbe74d',name='Precio unitario promedio'),secondary_y=True)
fig.update_layout(title_text='Número de ofertas por día de la semana', title_x=0.5,width=800,height=600,xaxis_title='Día',yaxis_title='Cantidad de ofertas')

In [176]:
vertical = ofertas_relampagos_df.groupby('VERTICAL')['OFFER_TYPE'].count().to_frame().rename(columns={'OFFER_TYPE':'Ofertas relámpago publicadas'}).join(ofertas_relampagos_df[ofertas_relampagos_df.SOLD_QUANTITY>0].groupby('VERTICAL')['OFFER_TYPE'].count().to_frame().rename(columns={'OFFER_TYPE':'Ofertas relámpago exitosas'}))
fig = go.Figure()
fig.add_trace(go.Bar(x=vertical.index,y=vertical['Ofertas relámpago publicadas'],name='Ofertas publicadas'))
fig.add_trace(go.Bar(x=vertical.index,y=vertical['Ofertas relámpago exitosas'],name='Ofertas exitosas'))

In [198]:
fig = go.Figure()
scatter_ofertas = ofertas_relampagos_df.groupby(['DURATION_HRS','VERTICAL'])['SOLD_QUANTITY'].sum().to_frame().reset_index()
color_map = dict(zip(ofertas_relampagos_df.VERTICAL.unique(),['#2E3273','#397FB9','#44CCFF','#5CB0F5','#7494EA','#A366D5','#D138BF','#E69086','#FBE74D']))
scatter_ofertas['COLOR'] = scatter_ofertas.VERTICAL.map(color_map)
for k in scatter_ofertas.VERTICAL.unique():
    df_filtrado = scatter_ofertas[scatter_ofertas.VERTICAL==k]
    fig.add_trace(go.Bar(x=df_filtrado.DURATION_HRS,y=df_filtrado.SOLD_QUANTITY,name=k,marker_color=df_filtrado.COLOR.values[0]))
fig.update_layout(title_text='Cantidades vendidas por duración de la oferta', title_x=0.5,width=800,height=600,xaxis_title='Duración (hrs)',yaxis_title='Cantidad vendida')

In [201]:
ofertas_relampagos_df

,OFFER_START_DATE,OFFER_START_DTTM,OFFER_FINISH_DTTM,OFFER_TYPE,INVOLVED_STOCK,REMAINING_STOCK_AFTER_END,SOLD_AMOUNT,SOLD_QUANTITY,ORIGIN,SHIPPING_PAYMENT_TYPE,DOM_DOMAIN_AGG1,VERTICAL,DOMAIN_ID,DURATION_HRS,DAY_OF_WEEK,PRICE_PER_UNIT
0,2021-06-22,2021-06-22 16:00:00+00:00,2021-06-22 23:02:43+00:00,lightning_deal,4,-2,4.72,6.0,A,none,PETS FOOD,CPG,MLM-BIRD_FOODS,7,Martes,0.79
1,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 19:00:02+00:00,lightning_deal,5,5,0.00,0.0,NaN,free_shipping,PET PRODUCTS,OTHERS,MLM-ANIMAL_AND_PET_PRODUCTS,6,Martes,0.00
2,2021-06-22,2021-06-22 07:00:00+00:00,2021-06-22 13:00:01+00:00,lightning_deal,15,12,10.73,3.0,NaN,none,COMPUTERS,CE,MLM-SPEAKERS,6,Martes,3.58
3,2021-06-22,2021-06-22 19:00:00+00:00,2021-06-23 01:36:12+00:00,lightning_deal,15,13,7.03,2.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,6,Martes,3.52
4,2021-06-22,2021-06-22 13:00:00+00:00,2021-06-22 15:48:12+00:00,lightning_deal,15,0,39.65,15.0,NaN,none,COMPUTERS,CE,MLM-HEADPHONES,2,Martes,2.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48741,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,15,9,16.28,6.0,NaN,none,HOME&DECOR,HOME & INDUSTRY,MLM-CHRISTMAS_LIGHTS,6,Sábado,2.71
48742,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:01+00:00,lightning_deal,5,5,0.00,0.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-DECORATIVE_PAINTINGS,6,Sábado,0.00
48743,2021-06-19,2021-06-19 07:00:00+00:00,2021-06-19 13:00:03+00:00,lightning_deal,5,3,16.62,2.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS,6,Sábado,8.31
48744,2021-06-19,2021-06-19 13:00:00+00:00,2021-06-19 19:00:00+00:00,lightning_deal,5,1,38.79,4.0,NaN,free_shipping,HOME&DECOR,HOME & INDUSTRY,MLM-INDOOR_CURTAINS_AND_BLINDS,6,Sábado,9.70


In [211]:
fig = go.Figure()
scatter_ofertas = ofertas_relampagos_df.groupby(['DURATION_HRS','DOM_DOMAIN_AGG1'])['SOLD_QUANTITY'].sum().to_frame().reset_index()

n_colors = len(ofertas_relampagos_df.DOM_DOMAIN_AGG1.unique())
cmap = mcolors.LinearSegmentedColormap.from_list("my_cmap", ['#2e3273', '#fbe74d'], N=n_colors)
colors = [mcolors.to_hex(cmap(i / (n_colors - 1))) for i in range(n_colors)]

color_map = dict(zip(ofertas_relampagos_df.DOM_DOMAIN_AGG1.unique(),colors))
scatter_ofertas['COLOR'] = scatter_ofertas.DOM_DOMAIN_AGG1.map(color_map)
for k in scatter_ofertas.DOM_DOMAIN_AGG1.unique():
    df_filtrado = scatter_ofertas[scatter_ofertas.DOM_DOMAIN_AGG1==k]
    fig.add_trace(go.Bar(x=df_filtrado.DURATION_HRS,y=df_filtrado.SOLD_QUANTITY,name=k,marker_color=df_filtrado.COLOR.values[0]))
fig.update_layout(title_text='Cantidades vendidas por duración de la oferta', title_x=0.5,width=800,height=600,xaxis_title='Duración (hrs)',yaxis_title='Cantidad vendida')

In [212]:
ofertas_relampagos_df.groupby(['SHIPPING_PAYMENT_TYPE','VERTICAL'])['SOLD_AMOUNT'].sum().to_frame().reset_index()

,SHIPPING_PAYMENT_TYPE,VERTICAL,SOLD_AMOUNT
0,free_shipping,ACC,18989.99
1,free_shipping,APP & SPORTS,93485.61
2,free_shipping,BEAUTY & HEALTH,165877.18
3,free_shipping,CE,336708.99
4,free_shipping,CPG,9811.86
5,free_shipping,ENTERTAINMENT,1579.73
6,free_shipping,HOME & INDUSTRY,109692.66
7,free_shipping,OTHERS,1346.53
8,free_shipping,T & B,11043.57
9,none,ACC,7829.99


In [216]:
fig = go.Figure()
shipping = ofertas_relampagos_df.groupby(['SHIPPING_PAYMENT_TYPE','VERTICAL'])['SOLD_AMOUNT'].mean().to_frame().reset_index()
fig.add_trace(go.Bar(x=shipping[shipping.SHIPPING_PAYMENT_TYPE=='free_shipping'].VERTICAL,y=shipping[shipping.SHIPPING_PAYMENT_TYPE=='free_shipping'].SOLD_AMOUNT,name='free_shipping',marker_color='#2e3273'))
fig.add_trace(go.Bar(x=shipping[shipping.SHIPPING_PAYMENT_TYPE=='none'].VERTICAL,y=shipping[shipping.SHIPPING_PAYMENT_TYPE=='none'].SOLD_AMOUNT,name='none',marker_color='#fbe74d'))
fig.update_layout(title_text='Sold amount por vertical según shipping time', title_x=0.5,width=800,height=600,xaxis_title='Vertical',yaxis_title='Sold amount $')